# Contents of This Program
- Demonstration of auto-calibartion -- just run this notebook, after changing setttings in ./program/04_calib_outsourced.py

In [1]:
PROGRAM_NAME = "06_calibration"

In [2]:
from quri_parts_oqtopus.backend import OqtopusSseBackend
from oqtopus_sse_pulse.adapter import QuriAdapter, collect_payloads_from_job, collect_session_from_job
from oqtopus_sse_pulse.plotters import plot_payload

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

DEVICE = "anemone" # デバイス名 (勉強会ではanemoneに固定)
backend = OqtopusSseBackend() # OqtopusSseBackendのインスタンスを作成

def experiment(program: str) -> list:
    """
    指定した実験プログラムを実行し, 測定結果のペイロードを返す関数.
    """

    # ジョブの実行
    job = backend.run_sse(
        file_path=f"program/{program}.py",
        device_id=f"{DEVICE}-pulse",
        name=program,
    )
    print(f"job_id = {job.job_id}")

    # ジョブの完了待ち
    job.wait_for_completion()
    print(f"job_status = {job.status}  elapsed={job.execution_time}")

    # QuriAdapterの作成
    adapter = QuriAdapter(backend)

    # 測定結果の取得
    payloads = collect_payloads_from_job(
        backend=adapter,
        job_id=job.job_id,
    )
    
    return payloads

In [3]:
payloads = experiment(PROGRAM_NAME)
print(payloads)

job_id = 069552a1-74eb-7e77-8000-61ad16b95eae
job_status = succeeded  elapsed=572.671
[{'calib_note': {'hpi_amplitude': {'Q36': 0.0273, 'Q37': 0.04}, 'rabi_params': {'Q36': {'target': 'Q36', 'frequency': 0.007787354493384955, 'amplitude': 2.114141859315592, 'phase': 0.012503583896865126, 'offset': -0.8330834966505071, 'noise': 0.016557656228542328, 'angle': 0.7904309789326529, 'distance': -1.3887121677398682, 'r2': 0.9988966753738866, 'reference_phase': -3.096436171449362, 'timestamp': '2025-12-31 14:14:40'}, 'Q38': {'target': 'Q38', 'frequency': 0.011163488624466705, 'amplitude': 1.6808228728102776, 'phase': -0.0026801836945621175, 'offset': -0.0042377431027766305, 'noise': 0.021088236942887306, 'angle': -0.6380569179865976, 'distance': 3.294478178024292, 'r2': 0.9991163784313247, 'reference_phase': -0.167581932845429, 'timestamp': '2025-12-31 14:15:01'}, 'Q39': {'target': 'Q39', 'frequency': 0.009792359855340223, 'amplitude': 0.689054287716553, 'phase': -0.03772726894312115, 'offset'

In [4]:
from pprint import pprint

pprint(payloads[0])

{'calib_note': {'hpi_amplitude': {'Q36': 0.0273, 'Q37': 0.04},
                'hpi_params': {'Q36': {'amplitude': 0.024198788975738794,
                                       'duration': 32.0,
                                       'target': 'Q36',
                                       'tau': 12,
                                       'timestamp': '2025-12-31 14:21:40'},
                               'Q38': {'amplitude': 0.06832916254834678,
                                       'duration': 32.0,
                                       'target': 'Q38',
                                       'tau': 12,
                                       'timestamp': '2025-12-31 14:21:47'},
                               'Q39': {'amplitude': 0.019308509271616394,
                                       'duration': 32.0,
                                       'target': 'Q39',
                                       'tau': 12,
                                       'timestamp': '2025-12-31 14:21:55'},

# calib_noteとpropsの書き出し

以下のブロックでcalib_noteとpropsの書き出しを行うので、必要な箇所を書き換えてpushする

In [ ]:
import json
import os
import yaml

if not os.path.exists("data"):
    os.makedirs("data")

with open(os.path.join("data", "calib_note.json"), "w") as f:
    json.dump(payloads[0].get("calib_note"), f, ensure_ascii=False, indent=2)
with open(os.path.join("data", "props.yaml"), "w", encoding="utf-8") as f:
    yaml.safe_dump(payloads[0].get("props"), f)
# with open(os.path.join("data", "calibration.json"), "w") as f:
#     json.dump(payloads[0], f, ensure_ascii=False, indent=2)